In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.cross_validation import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#lines_eng= pd.read_table('C:\\Users\\richa\\Desktop\\conversica\\data\\set1\\training.hi-en.en.txt', names=['eng'])

In [3]:
#lines_hin=pd.read_table('C:\\Users\\richa\\Desktop\\conversica\\data\\set1\\training.hi-en.hi.txt',names=['hin'])

In [39]:
lines = pd.read_table('C:\\Users\\richa\\Desktop\\conversica\\fra.txt',names=['eng','hin'])

In [4]:
#len(lines_hin),len(lines_eng)

(35329, 37726)

In [5]:
#lines_hin=lines_hin[0:1001]
#lines_eng=lines_eng[0:1001]

In [6]:
#lines =pd.DataFrame()
#lines['eng']=lines_eng
#lines['hin']=lines_hin

In [40]:
lines.head(3)

,eng,hin
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !


In [41]:
lines.shape

(149861, 2)

In [42]:
lines = lines[0:10000]

In [9]:
### data preprocessing

In [43]:
## converting to lowercase
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())

In [44]:

lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [45]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [46]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hin=lines.hin.apply(lambda x: x.translate(remove_digits))

In [47]:
lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [48]:
### create set of all english and french words
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [49]:
len(all_eng_words), len(all_hindi_words)

(2146, 4567)

In [50]:
## finding the longest sentence and its length, hindi
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

13

In [51]:
## finding the longest sentence and its length, english
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

5

In [52]:
## soritng the list of all english and hindi words and then finding the length for encoding
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)

In [53]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [54]:
len(lines.hin)*13*num_decoder_tokens

593710000

In [55]:
encoder_input_data = np.zeros((len(lines.eng), 5),dtype='float32')
decoder_input_data = np.zeros((len(lines.hin), 13),dtype='float32')
decoder_target_data = np.zeros((len(lines.hin), 13, num_decoder_tokens),dtype='float32')

In [56]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.hin)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [57]:
embedding_size = 50


In [58]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [59]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [60]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=20,
          validation_split=0.05)

Train on 9500 samples, validate on 500 samples
Epoch 1/20
9500/9500 [==============================] - 144s 15ms/step - loss: 2.0565 - acc: 0.0757 - val_loss: 2.1972 - val_acc: 0.0769
Epoch 2/20
9500/9500 [==============================] - 137s 14ms/step - loss: 1.7355 - acc: 0.0769 - val_loss: 2.1949 - val_acc: 0.0769
Epoch 3/20
9500/9500 [==============================] - 132s 14ms/step - loss: 1.6666 - acc: 0.0778 - val_loss: 2.0982 - val_acc: 0.1062
Epoch 4/20
9500/9500 [==============================] - 138s 15ms/step - loss: 1.5809 - acc: 0.0917 - val_loss: 2.0416 - val_acc: 0.1077
Epoch 5/20
9500/9500 [==============================] - 132s 14ms/step - loss: 1.5151 - acc: 0.0963 - val_loss: 1.9833 - val_acc: 0.1092
Epoch 6/20
9500/9500 [==============================] - 135s 14ms/step - loss: 1.4642 - acc: 0.0988 - val_loss: 1.9782 - val_acc: 0.1123
Epoch 7/20
9500/9500 [==============================] - 132s 14ms/step - loss: 1.4231 - acc: 0.1043 - val_loss: 1.9310 - val_acc: 0

ResourceExhaustedError: OOM when allocating tensor with shape[128,13,4567] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[Node: loss_1/dense_2_loss/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_dense_2_target_0_1, loss_1/dense_2_loss/Log)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'loss_1/dense_2_loss/mul', defined at:
  File "C:\ProgramData\Anaconda3\envs\py35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\traitlets\config\application.py", line 658, in launch_in